## 0. Import dependencies

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sys
sys.path.insert(0, '../utils')
from load_data import ames_housing

import functools
import tensorflow as tf
from tensorflow.python.feature_column.feature_column import _LazyBuilder
tf.__version__

'1.9.0'

## 1. Load data

In [2]:
dirfolder = '../data/Ames-Housing'
numvalid = 100
train, valid, testdf, listnumeric, dictcategorical = ames_housing(dirfolder, numvalid)
traindf, trainy = train
validdf, validy = valid

Number of train samples is 1360.
Number of valid samples is 100.
Number of test samples is 1459.
Number of feature columns is 79.
Checking listnumeric ...
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
Checking dictcategorical ...
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


/home/ymtoo/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/ymtoo/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## 2. Setting TensorFlow input data

In [ ]:
def feature_column(listnumeric, dictcategorical):
    listcategorical = list(dictcategorical.keys())
    numeric_cols = []
    for num in listnumeric:
        numeric_cols.append(tf.feature_column.numeric_column(num))
    categorical_cols = []
    for cat in listcategorical:
        cattemp = tf.feature_column.categorical_column_with_vocabulary_list(
                cat, dictcategorical[cat])
        categorical_cols.append(cattemp)
    return numeric_cols, categorical_cols

numeric_cols , categorical_cols = feature_column(listnumeric, dictcategorical)
indicator_cols = list(map(tf.feature_column.indicator_column, categorical_cols))

## 3. Regression using deep neural network

In [ ]:
def easy_input_fn(df, label, num_epochs, shuffle, batch_size):
    if label is None:
        inputs = dict(df)
    else:
        inputs = (dict(df), label)
    ds = tf.data.Dataset.from_tensor_slices(inputs)
    
    if shuffle:
        ds = ds.shuffle(10000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds

train_inpf =  functools.partial(easy_input_fn, 
                                traindf,
                                label=trainy,  
                                num_epochs=100, 
                                shuffle=True, 
                                batch_size=64)

valid_inpf =  functools.partial(easy_input_fn, 
                                validdf, 
                                label=validy,
                                num_epochs=1, 
                                shuffle=True, 
                                batch_size=64)

test_inpf =  functools.partial(easy_input_fn, 
                               validdf, 
                               label=None,
                               num_epochs=1, 
                               shuffle=True, 
                               batch_size=64)

reg = tf.estimator.DNNRegressor(
    feature_columns=numeric_cols+indicator_cols,
    hidden_units=[1024, 512, 256],
    optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.01,
      l1_regularization_strength=0.001))
reg.train(train_inpf)
result = reg.evaluate(valid_inpf)

for key,value in sorted(result.items()):
    print('%s: %0.2f' % (key, value))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpfxxwduhd', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5715873048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpfxxwduhd/model.ckpt.
INFO:ten

In [ ]:
pred = reg.predict(valid_inpf)
validypred = np.zeros((validy.size))
for i, p in enumerate(pred):
    validypred[i] = p['predictions'][0]

## 4. Plot result

In [ ]:
fig, ax = plt.subplots()
ax.plot(validy, label='actual')
ax.plot(validypred, label='predict')
ax.set_xlabel('Valid data index')
ax.grid()
ax.legend()